### **_Importar librerias_**

In [1]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [2]:
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth', None)

### **_Lectura de datos_**

In [3]:
redpiso_normal= pd.read_excel("../01-datos-iniciales/06-RedPiso/Redpiso.xlsx")
redpiso_detalles= pd.read_excel("../01-datos-iniciales/06-RedPiso/Detalle_RedPiso.xlsx")
dbcp_ordenado= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

dfcalles= pd.read_excel('../02-datos-limpios/CALLEJERO_VIGENTE_NUMERACIONES_202501.xlsx')

In [4]:

#dbcp_ordenado= pd.read_excel("distr_barrios_codpostal.xlsx")
#redpiso_normal= pd.read_excel("Redpiso.xlsx")
#redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [5]:
def extraer_calle(titulo):
    palabras_clave = ["CALLE", "AVENIDA", "PLAZA","PASEO"]
    palabras = titulo.split(",")  # Dividir por comas
    for segmento in palabras:
        for palabra in palabras_clave:
            if palabra in segmento.upper():
                indice = segmento.upper().find(palabra)
                return segmento[indice:].strip()  # Extraer desde la palabra clave hasta la coma siguiente
    return None

In [6]:
def extraer_barrio_distrito(titulo):
    partes = [p.strip() for p in titulo.split(",")]
    if len(partes) >= 2:
        return partes[-2], partes[-1]  # Última parte es el distrito, la anterior es el barrio
        # busco si se identifica el barrio con el que aparece en la columna barrio de dbcp_ordenado
    return None, None



In [7]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 

def limpiar_texto(x):
    return re.sub(r'[^\w\s]','',unidecode(str(x)))



In [8]:
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor 

In [9]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

### **_Aqui se termina la union de los dos ficheros de pisos madrid y empezamos con redpiso_**

----------

In [10]:
## Aqui empieza el primer tratamiento de redpiso normal

In [11]:
redpiso_normal.columns

Index(['Título', 'Imagen', 'item', 'item1', 'propertylistoptionsitememblem',
       'propertylistoptionsitem', 'propertylistoptionsitem2',
       'propertylistoptionsitem3', 'itemlink_URL', 'itemlink', 'Imagen4',
       'Número', 'Número5', 'officelink_URL', 'officelink'],
      dtype='object')

In [12]:
# Extraer la última palabra después del guion 

redpiso_normal["referencia"] = redpiso_normal['itemlink_URL'].apply(lambda x: re.findall(r'[^-]+$', x)[0])

In [13]:
# Conversion de letras a minusculas en las columnas
redpiso_normal.columns= redpiso_normal.columns.str.lower()

In [14]:
# Columnas que seran eliminadas ya previamente analizadas.
columnas_a_eliminar = ["item","item1","imagen4","propertylistoptionsitem3","número5"]

redpiso_normal= redpiso_normal.drop(columns=columnas_a_eliminar)

In [15]:
# Renombramiento de la columna.
redpiso_normal = redpiso_normal.rename(columns={"título": "precio"})
redpiso_normal = redpiso_normal.rename(columns={"imagen": "url_imagen"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitememblem": "reservado"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem": "m2_constr"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem2": "cantidad_habitaciones"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink_url": "url"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink": "descripcion"})
redpiso_normal = redpiso_normal.rename(columns={"número": "contacto"})
redpiso_normal = redpiso_normal.rename(columns={"officelink": "nombre_oficina"})

In [16]:
# quitamos la palabra tel: y solo dejamos los valores numeros 
redpiso_normal["contacto"] = redpiso_normal["contacto"].str.replace(r'(?i)^tel:', "", regex=True)

In [17]:
# Cree una funcion que que toma los espacios y los caracteres especiales sin dejar algo detras.
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor

# Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
#redpiso_normal= redpiso_normal.apply(lambda col: col.apply(limpiar_valores))

In [18]:
#20022024 comento la limpieza, no quiero que quites puntos y comas
#for col in redpiso_normal.columns: 
 #   if col != 'url': 
  #      redpiso_normal[col] = redpiso_normal[col].apply(limpiar_valores) 
     

In [19]:
# Rellena los valores nulos a sin_reserva
#redpiso_normal["reservado/sin_reserva"]= redpiso_normal["reservado/sin_reserva"].fillna("sin_reserva")

In [20]:
# Expresion regular que recorre la columna extrae los numeros de la cada valor y elimina lo demas.aplicada a la columna que se muestra.

redpiso_normal["cantidad_habitaciones"] = redpiso_normal["cantidad_habitaciones"].str.extract(r'(\d+)')

redpiso_normal["m2_constr"] = redpiso_normal["m2_constr"].str.extract(r'(\d+)')

In [21]:


# Asignar 1 a "Reservado", 0 a "Sin Reserva" y dejar el resto como están
redpiso_normal["reservado"] = redpiso_normal["reservado"].apply(lambda x: 1 if str(x).strip().upper() == "RESERVADO" else 0)




In [22]:
# Se convierte los valores RESERVADO a 1 y todo los demas a 0, basicamente los que aun no estan reservados seran 0.
#redpiso_normal["reservado/sin_reserva"] = redpiso_normal ["reservado/sin_reserva"].apply(lambda x : 1 if x.strip().upper()== "RESERVADO" else 0)

In [23]:
redpiso_normal.isna().sum()

precio                     0
url_imagen               723
reservado                  0
m2_constr                  7
cantidad_habitaciones      5
url                        0
descripcion                0
contacto                   0
officelink_url             0
nombre_oficina             0
referencia                 0
dtype: int64

In [24]:
redpiso_normal.shape

(2108, 11)

### **_Aqui se termina el tratamiento de redpiso_normal,ahora sigue el procesmiento de redpiso_detalles_**

In [25]:
#redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [26]:
redpiso_detalles.head(3)

,Título,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Tiempo,Tiempo1,Campo7,Campo8,Campo92,Campo9,Campo10,Campo11,Campo12,Unnamed: 16,Campo13,Campo131,Tiempo2,Campo14,Campo15,Tiempo3,Campo16,Campo17,Campo18,Tiempo4,Campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde, Madrid, Madrid",106.500 €,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde, Madrid, Madrid\n 106.500 €\n\n ESTUPENDA VIVIENDA EN CALLE BENIMAMET\n \n RP2842024125161\n \n \n \n \n Concertar una visita \n Añadir a favoritos\n \n Conseguir financiación",ESTUPENDA VIVIENDA EN CALLE BENIMAMET,RP2842024125161,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde, Madrid, Madrid",REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,2.0,1.0,1,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito en el RAIN: 000517/20. En venta este inmueble de 64m² , ubicado en una 5º planta . Consta de 2 amplios dormitorios ( antes 3) con armarios empotrados, salón y cocina independientes, baño con plato de ducha . Además cuenta con aire acondicionado en salón , suelos de tarima .Muy luminoso!. PARA ENTRAR A VIVIR, NO DEJES PASAR ESTA OPORTUNIDAD. San Cristóbal de los Ángeles es un barrio del extrarradio de Madrid, humilde y lleno de vida. Rodeado de grandes zonas verdes, como nuestro precioso parque, conocido por su estanque. Un lugar ideal para pasear, hacer deporte o celebraciones al aire libre. La zona cuenta con comercios muy variados, un amplio mercado y dos supermercados, además de zonas para practicar deporte, como son una cancha de baloncesto, pistas donde poder jugar al tenis y un gran campo de futbol donde los pequeños pueden practicar y aprender su deporte favorito. También cuenta con varios colegios cercanos y un centro de educación secundaria. El barrio tiene muy buena comunicación, dos líneas de autobuses de la EMT (59 Y 79) y Metro ( Linea 3) parada San Cristóbal de los Ángeles, a 15 minutos de Sol. También contamos con parada de Cercanías RENFE, tan solo a 11 minutos de la estación de Atocha y 14 minutos de Sol. Carretera salida a la avenida de Andalucía con salidas próximas a la M30, M40, M45, M50. En el barrio se encuentran situados el Centro de Salud San Cristóbal y el Hospital de día Villaverde. El hospital 12 de Octubre está a diez minutos.",Metros: 64 m²,NaN,Metros útiles: 60 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,"Piso en venta en Usera, Madrid, Madrid\n 119.000 €\n\n ESTUDIO JARDINES DE YBARRA II\n \n RP782024130863\n \n \n Concertar una visita \n Añadir a favoritos\n \n Conseguir financiación",ESTUDIO JARDINES DE YBARRA II,RP782024130863,"Piso en venta en Usera, Madrid, Madrid",REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,1.0,1,ESTUDIO JARDINES DE YBARRA II,"Redpiso vende en exclusiva ACOGEDOR ESTUDIO en Jardines de Ybarra II, en el Poblado de Orcasitas. Consta de salón (dormitorio), cocina americana y baño con plato de ducha. Calefacción individual eléctrica, agua caliente por termo. Situado en zona muy tranquila y consolidada con todo tipo de comercios y servicios cercanos (Mercadona, centro de salud, instalaciones deportivas, colegios...). Comunicado con varias líneas EMT, Renfe de Orcasitas y accesos rápidos a A42 y M40. Fácil aparcamiento en la zona. Visítalo sin compromiso.",Metros: 30 m²,NaN,Metros útiles: 27 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madrid, Madrid",120.000 €,"Piso en venta en Buenavista, Carabanchel, Madrid, Madrid\n 120.000 €\n\n ¡Tu nuevo hogar te espera en Buena Vista!\n \n RP442024126007\n \n \n \n \n Concertar una visita \n Añadir a favoritos\n \n Conseguir financiació

# renombramiento de columnas 
redpiso_detalles = redpiso_detalles.rename(columns={"Título":"titulo"})
redpiso_detalles = redpiso_detalles.rename( columns={"Campo1": "precio"})
#redpiso_detalles = redpiso_detalles.rename(columns={"Campo2":"descripcion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo":"cantidad_visitas"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
#redpiso_detalles = redpiso_detalles.rename(columns={"Campo7":"espacio_metros_cuadrados"})
#redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo4": "referencia"})
#redpiso_detalles = redpiso_detalles.rename( columns={"título": "titulo"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo7":"m2_constr"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo13":"metros_utiles"})
redpiso_detalles = redpiso_detalles.rename(columns={"campo8": "dormitorios"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo9": "banos"})
#redpiso_detalles = redpiso_detalles.rename(columns={"campo15": "cant_armarios"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo14":"calidad_energetica"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo10":"titularanuncio"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo11":"descripcion_larga"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo4":"referencia"})

In [27]:
redpiso_detalles = redpiso_detalles.rename(columns={
    "Título": "titulo",
    "Campo1": "precio",
    "Tiempo": "cantidad_visitas",
    "Tiempo1": "fecha_publicacion",
    "Campo4": "referencia",
    "Campo7": "m2_constr",
    "Campo13": "m2_utiles",
    "Campo8": "dormitorios",
    "Campo9": "banos",
    "Campo10": "titularanuncio",
    "Campo11": "descripcion_larga"
})


In [28]:
redpiso_detalles.columns

Index(['titulo', 'precio', 'Campo2', 'Campo3', 'referencia', 'Campo5',
       'Campo6', 'cantidad_visitas', 'fecha_publicacion', 'm2_constr',
       'dormitorios', 'Campo92', 'banos', 'titularanuncio',
       'descripcion_larga', 'Campo12', 'Unnamed: 16', 'm2_utiles', 'Campo131',
       'Tiempo2', 'Campo14', 'Campo15', 'Tiempo3', 'Campo16', 'Campo17',
       'Campo18', 'Tiempo4', 'Campo19'],
      dtype='object')

In [29]:
#dfrdetalles["titulo"] = dfrdetalles["titulo"].str.rstrip(", Madrid, Madrid")

In [30]:
# eliminacion de columnas
#redpiso_detalles= redpiso_detalles.drop("descripcion", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo5", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo9",axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo10",axis=1)
#redpiso_detalles = redpiso_detalles.drop ("Campo11", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo12",axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo8", axis= 1)
#redpiso_detalles = redpiso_detalles.drop("Campo3", axis= 1)
#redpiso_detalles = redpiso_detalles.drop("Campo4", axis= 1)

In [31]:
# Columnas a minusculas
redpiso_detalles.columns = redpiso_detalles.columns.str.lower()

In [32]:
# Eliminacion de las primeras 3 letras en la referencia web - REF- dejando solo la referencia 
#redpiso_detalles["referencia_web"] = redpiso_detalles["referencia_web"].str[3:].str.strip()

In [33]:
#20022024 ecomento la eliminacion de todos los caracteres especiales sin discriminacion d la columna

# Reemplace todas las letras ñ por n.
#redpiso_detalles = redpiso_detalles.apply(lambda col : col.str.replace("ñ","n")if col.dtype == "object" else col)



#  Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
#redpiso_detalles = redpiso_detalles.apply(lambda col: col.apply(limpiar_valores))


# Cree una funcion que elimina todos los acentos  y caracteres especiales
#redpiso_detalles = redpiso_detalles.apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y,str) else y )if x.dtype == "object" else x)



In [34]:
def crear_columnas(df,nueva_col,valor_busqueda):
   # df[nueva_col]= np.nan
    for index, row in df.iterrows():
     for col in df.columns[:-1]:
        if pd.notna(row[col]):
            for valor in valor_busqueda:
                if valor in str(row[col]):
                    df.at[index, nueva_col] =valor 
                    #df.at[index, col]= np.nan
                    break


In [35]:
valores_buscar = [
    "primera planta", "Primera planta", "1ª planta", "1a planta",
    "segunda planta", "Segunda planta", "2ª planta", "2a planta",
    "tercera planta", "Tercera planta", "3ª planta", "3a planta",
    "cuarta planta", "Cuarta planta", "4ª planta", "4a planta",
    "quinta planta", "Quinta planta", "5ª planta", "5a planta",
    "sexta planta", "Sexta planta", "6ª planta", "6a planta",
    "séptima planta", "Séptima planta", "7ª planta", "7a planta",
    "octava planta", "Octava planta", "8ª planta", "8a planta",
    "novena planta", "Novena planta", "9ª planta", "9a planta",
    "décima planta", "Décima planta", "10ª planta", "10a planta",
    "undécima planta", "Undécima planta", "11ª planta", "11a planta",
    "duodécima planta", "Duodécima planta", "12ª planta", "12a planta",
    "decimotercera planta", "Decimotercera planta", "13ª planta", "13a planta",
    "decimocuarta planta", "Decimocuarta planta", "14ª planta", "14a planta",
    "decimoquinta planta", "Decimoquinta planta", "15ª planta", "15a planta",
    "decimosexta planta", "Decimosexta planta", "16ª planta", "16a planta",
    "decimoséptima planta", "Decimoséptima planta", "17ª planta", "17a planta",
    "decimoctava planta", "Decimoctava planta", "18ª planta", "18a planta",
    "decimonovena planta", "Decimonovena planta", "19ª planta", "19a planta",
    "vigésima planta", "Vigésima planta", "20ª planta", "20a planta"
]



crear_columnas(redpiso_detalles, "planta", valores_buscar)

# Creacion de columnas nuevas y rellenarlos con nulos.  Despues iterar sobre las columnas y rellenarlas con los valores declararlos en las listas de atras
valores_buscar1= ["Dormitorios 1", "Dormitorios 2", "Dormitorios 3", "Dormitorios 4","Dormitorios1", "Dormitorios2","Dormitorios3", "Dormitorios4",
                  "Dormitorios 5", "Dormitorios 6", "Dormitorios 7", "Dormitorios 8","Dormitorios  8", "Dormitorios 9", "Dormitorios 10", "Dormitorios5","Dormitorios6",
                  "Dormitorios7", "Dormitorios8", "Dormitoros9"]

valores_buscar2= ["Banos 1", "Banos 2", "Banos 3", "Banos 4","Banos 5","Banos 6","Banos 7","Banos 8",
                   "Banos1", "Banos2", "Banos3", "Banos4", "Banos5", "Banos6", "Banos7","Banos8"]

nueva_columna = "dormitorios"
nueva_columna2 = "banos"


#redpiso_detalles[nueva_columna]= np.nan
#redpiso_detalles[nueva_columna2]= np.nan

#for index, row in redpiso_detalles.iterrows():
 #   for col in redpiso_detalles .columns[:-1]:
  #      if pd.notna(row[col]):
   #         for valor in valores_buscar1:
    #            if valor in str(row[col]):
     #               redpiso_detalles.at[index, nueva_columna] =valor 
      #              redpiso_detalles.at[index, col]= np.nan
       #             break




for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles .columns[:-1]:
        if pd.notna(row[col]):
            for valor in valores_buscar2:
                if valor in str(row[col]):
                    redpiso_detalles.at[index, nueva_columna2] =valor 
                    redpiso_detalles.at[index, col]= np.nan
                    break

In [36]:


# Función para crear y rellenar la nueva columna en base a un patrón
def crear_columnas_conpatron(df, nueva_col, patron):
    df[nueva_col] = np.nan  # Inicializar la nueva columna con valores NaN si no existe
    for index, row in df.iterrows():
        #for col in df.columns[:-1]:  # Excluir la nueva columna en la iteración
        for col in df.columns:
            if pd.notna(row[col]):
                valor = str(row[col])
                if pd.Series(valor).str.contains(patron, regex=True).any():
                    df.at[index, nueva_col] = row[col]
                    # df.at[index, col] = np.nan  # Descomentar esta línea si quieres establecer NaN en el valor original
                    break  # Salir del bucle interno una vez que se ha encontrado el valor y asignado

In [37]:
patron = r'(?i)^orientación:'
crear_columnas_conpatron(redpiso_detalles,"orientacion",patron)

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\2179494747.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Orientación: Sur' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, nueva_col] = row[col]


In [38]:
#patron = r'(?i)^baños' 
#crear_columnas_conpatron(redpiso_detalles,"banos",patron)

In [39]:
patron = r'(?i)^Año de construcción:'
crear_columnas_conpatron(redpiso_detalles,"ano_construccion",patron)

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\2179494747.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Año de construcción: 1962' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, nueva_col] = row[col]


In [40]:

nueva_columna3= "armarios"
patron = r'(?i)^armario' 
redpiso_detalles[nueva_columna3]= np.nan

redpiso_detalles[nueva_columna3] = redpiso_detalles[nueva_columna3].astype("object") # Conversion de columna float64 a object.

# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron, regex=True).any():
                redpiso_detalles.at[index, nueva_columna3]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [41]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna4 = "consumo_energia"
patron_2 = r'(?i)^consumo'
redpiso_detalles[nueva_columna4]= np.nan
redpiso_detalles[nueva_columna4] = redpiso_detalles[nueva_columna4].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_2, regex=True).any():
                redpiso_detalles.at[index, nueva_columna4]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [42]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna5 = "emisiones_CO2"
patron_3 = r'(?i)^emisiones'
redpiso_detalles[nueva_columna5]= np.nan
redpiso_detalles[nueva_columna5] = redpiso_detalles[nueva_columna5].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_3, regex=True).any():
                redpiso_detalles.at[index, nueva_columna5]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna6 = "cal_energetica"
patron_4 = r'(?i)^cal'
redpiso_detalles[nueva_columna6]= np.nan
redpiso_detalles[nueva_columna6] = redpiso_detalles[nueva_columna6].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_4, regex=True).any():
                redpiso_detalles.at[index, nueva_columna6]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [43]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna7="estado"
patron_5 =  r'(?i)^estado'
redpiso_detalles[nueva_columna7]= np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_5, regex=True).any():
                redpiso_detalles.at[index, nueva_columna7]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1235483781.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Estado: Perfecto Estado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna7]= row[col]


In [44]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna8 = "antiguedad"
patron_6= r'(?i)^Antigüedad'
redpiso_detalles[nueva_columna8]= np.nan
redpiso_detalles[nueva_columna8] = redpiso_detalles[nueva_columna8].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_6, regex=True).any():
                redpiso_detalles.at[index, nueva_columna8]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [45]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna9 = "ascensor"
patron_7= r'(?i)^ascensor'
redpiso_detalles[nueva_columna9]= np.nan
redpiso_detalles[nueva_columna9] = redpiso_detalles[nueva_columna9].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_7, regex=True).any():
                redpiso_detalles.at[index, nueva_columna9]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [46]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna10 = "aire_acondicionado"
patron_8= r'(?i)^aire'
redpiso_detalles[nueva_columna10]= np.nan
redpiso_detalles[nueva_columna10] = redpiso_detalles[nueva_columna10].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_8, regex=True).any():
                redpiso_detalles.at[index, nueva_columna10]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna11 = "orientacion"
patron_9= r'(?i)^orientacion'
redpiso_detalles[nueva_columna11]= np.nan
redpiso_detalles[nueva_columna11] = redpiso_detalles[nueva_columna11].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_9, regex=True).any():
                redpiso_detalles.at[index, nueva_columna11]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [47]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna12 = "agua_caliente"
patron_10= r'(?i)^agua'
redpiso_detalles[nueva_columna12]= np.nan
redpiso_detalles[nueva_columna12] = redpiso_detalles[nueva_columna12].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_10, regex=True).any():
                redpiso_detalles.at[index, nueva_columna12]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [48]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna13 = "calefaccion"
patron_12= r'(?i)^Calefacción'
redpiso_detalles[nueva_columna13]= np.nan
redpiso_detalles[nueva_columna13] = redpiso_detalles[nueva_columna13].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_12, regex=True).any():
                redpiso_detalles.at[index, nueva_columna13]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [49]:
redpiso_detalles["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                            1288
Calefacción: Gas Natural        311
Calefacción: Individual         189
Calefacción: Eléctrica          165
Calefacción: Central            148
Calefacción: Suelo Radiante       5
Calefacción: Aerotermia           1
Calefacción: Propano              1
Name: count, dtype: int64

In [50]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna14 = "tipo_suelo"
patron_13= r'(?i)^tipo de suelo'
redpiso_detalles[nueva_columna14]= np.nan
redpiso_detalles[nueva_columna14] = redpiso_detalles[nueva_columna14].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_13, regex=True).any():
                redpiso_detalles.at[index, nueva_columna14]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

nueva_columna14 = "tipo_fachada"
patron_13= r'(?i)^tipo de fachada'
redpiso_detalles[nueva_columna14]= np.nan
redpiso_detalles[nueva_columna14] = redpiso_detalles[nueva_columna14].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_13, regex=True).any():
                redpiso_detalles.at[index, nueva_columna14]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


# patron_14_tipo= r'^\S*tipo' #  Asegura que no haya espacios antes de palabra. Expresion regular.
# # Aqui se ejerce la busqueda de tipo pero reconociendo espacios antes de la palabra. Despues de coloca en la columna tipo de suelo_fachada.
# for index, row in datos.iterrows():
#     for col in datos.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_14_tipo, regex=True, flags=re.IGNORECASE).any():
#                 datos.at[index,nueva_columna14]= row[col]
#                 datos.at[index, col]= np.nan
#                 break

In [51]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna15 = "cocina_equipada"
patron_15= r'(?i)^cocina'


redpiso_detalles[nueva_columna15]= np.nan
redpiso_detalles[nueva_columna15] = redpiso_detalles[nueva_columna15].astype("object")
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_15, regex=True).any():
                redpiso_detalles.at[index, nueva_columna15]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [52]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna16 = "garaje"
patron_16= r'(?i)^garajes'
redpiso_detalles[nueva_columna16]= np.nan
redpiso_detalles[nueva_columna16] = redpiso_detalles[nueva_columna16].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_16, regex=True).any():
                redpiso_detalles.at[index, nueva_columna16]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [53]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_20= r'(?i)^exterior'
nueva_columna18 = "exterior"
redpiso_detalles[nueva_columna18] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_20, regex=True).any():
                redpiso_detalles.at[index, nueva_columna18]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


C:\Users\extas\AppData\Local\Temp\ipykernel_11376\2400203449.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Exterior' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna18]= row[col]


In [54]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_21= r'(?i)^piscina'
nueva_columna19 = "tiene_piscina"
redpiso_detalles[nueva_columna19] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_21, regex=True).any():
                redpiso_detalles.at[index, nueva_columna19]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\807033423.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Piscina' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna19]= row[col]


In [55]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_22= r'(?i)^trastero'
nueva_columna22 = "trastero"
redpiso_detalles[nueva_columna22] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_22, regex=True).any():
                redpiso_detalles.at[index, nueva_columna22]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\4053109142.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trastero' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna22]= row[col]


In [56]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_23= r'(?i)^comunidad'
nueva_columna23 = "gastos_comunidad"
redpiso_detalles[nueva_columna23] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_23, regex=True).any():
                redpiso_detalles.at[index, nueva_columna23]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\2512737850.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Comunidad: 74,00 €' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna23]= row[col]


In [57]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_24= r'(?i)^zonas'
nueva_columna24 = "zonas_verdes"
redpiso_detalles[nueva_columna24] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_24, regex=True).any():
                redpiso_detalles.at[index, nueva_columna24]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1602726678.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Zonas verdes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna24]= row[col]


In [58]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_25= r'(?i)^terraza'
nueva_columna25 = "terraza"
redpiso_detalles[nueva_columna25] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_25, regex=True).any():
                redpiso_detalles.at[index, nueva_columna25]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [59]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_26= r'(?i)^acceso'
nueva_columna26 = "acceso_minusvalido"
redpiso_detalles[nueva_columna26] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_26, regex=True).any():
                redpiso_detalles.at[index, nueva_columna26]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\3499825224.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Acceso minusválidos' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna26]= row[col]


In [60]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_27= r'(?i)^puerta'
nueva_columna27 = "puerta_seguridad"
redpiso_detalles[nueva_columna27] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_27, regex=True).any():
                redpiso_detalles.at[index, nueva_columna27]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1113838515.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Puerta de seguridad' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna27]= row[col]


In [61]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_28= r'(?i)^conserje'
nueva_columna28 = "porteria"
redpiso_detalles[nueva_columna28] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_28, regex=True).any():
                redpiso_detalles.at[index, nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\4262138081.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Conserje' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna28]= row[col]


In [62]:
# Aqui se filtra la palabra Conserje pero sin distincion de espacios antes de la palabra. 
patron_29_conserje= r'^\S*Conserje'
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_29_conserje, regex=True, flags=re.IGNORECASE).any():
                redpiso_detalles.at[index,nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [63]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_30= r'(?i)^plantas'

nueva_columna29 = "num_plantas"

redpiso_detalles[nueva_columna29] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_30, regex=True).any():
                redpiso_detalles.at[index, nueva_columna29]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\757634582.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Plantas: 5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna29]= row[col]


In [64]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_31= r'(?i)^pisos'

nueva_columna30 = "num_pisos"

redpiso_detalles[nueva_columna30] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_31, regex=True).any():
                redpiso_detalles.at[index, nueva_columna30]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1093162397.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Pisos: 25' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna30]= row[col]


In [65]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_32= r'(?i)^alarma'

nueva_columna31 = "alarma"

redpiso_detalles[nueva_columna31] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_32, regex=True).any():
                redpiso_detalles.at[index, nueva_columna31]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1007444481.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Alarma' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna31]= row[col]


In [66]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_33= r'(?i)^amueblado'

nueva_columna32 = "amueblado"

redpiso_detalles[nueva_columna32] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_33, regex=True).any():
                redpiso_detalles.at[index, nueva_columna32]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1403479313.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Amueblado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna32]= row[col]


In [67]:
redpiso_detalles.columns

Index(['titulo', 'precio', 'campo2', 'campo3', 'referencia', 'campo5',
       'campo6', 'cantidad_visitas', 'fecha_publicacion', 'm2_constr',
       'dormitorios', 'campo92', 'banos', 'titularanuncio',
       'descripcion_larga', 'campo12', 'unnamed: 16', 'm2_utiles', 'campo131',
       'tiempo2', 'campo14', 'campo15', 'tiempo3', 'campo16', 'campo17',
       'campo18', 'tiempo4', 'campo19', 'planta', 'orientacion',
       'ano_construccion', 'armarios', 'consumo_energia', 'emisiones_CO2',
       'estado', 'antiguedad', 'ascensor', 'aire_acondicionado',
       'agua_caliente', 'calefaccion', 'tipo_suelo', 'tipo_fachada',
       'cocina_equipada', 'garaje', 'exterior', 'tiene_piscina', 'trastero',
       'gastos_comunidad', 'zonas_verdes', 'terraza', 'acceso_minusvalido',
       'puerta_seguridad', 'porteria', 'num_plantas', 'num_pisos', 'alarma',
       'amueblado'],
      dtype='object')

In [68]:

dfrdetalles = redpiso_detalles[[
    "titulo", "precio", "cantidad_visitas", "fecha_publicacion", "referencia", 
    "m2_constr", "m2_utiles", "dormitorios", "banos", "titularanuncio", 
    "descripcion_larga", "amueblado", "alarma", "num_pisos", "num_plantas", 
    "puerta_seguridad", "acceso_minusvalido", "porteria", "terraza", "zonas_verdes", 
    "planta", "garaje", "trastero", "gastos_comunidad", "tiene_piscina", "exterior", 
    "tipo_fachada", "tipo_suelo", "calefaccion", "aire_acondicionado", "antiguedad", 
    "ascensor", "agua_caliente", "estado", "emisiones_CO2", 
    "consumo_energia", "armarios", "orientacion","ano_construccion"
]]






In [69]:
# Mapeo de valores textuales a valores numéricos
mapeo_plantas = {
    "primera planta": 1, "Primera planta": 1, "1ª planta": 1, "1a planta": 1,
    "segunda planta": 2, "Segunda planta": 2, "2ª planta": 2, "2a planta": 2,
    "tercera planta": 3, "Tercera planta": 3, "3ª planta": 3, "3a planta": 3,
    "cuarta planta": 4, "Cuarta planta": 4, "4ª planta": 4, "4a planta": 4,
    "quinta planta": 5, "Quinta planta": 5, "5ª planta": 5, "5a planta": 5,
    "sexta planta": 6, "Sexta planta": 6, "6ª planta": 6, "6a planta": 6,
    "séptima planta": 7, "Séptima planta": 7, "7ª planta": 7, "7a planta": 7,
    "octava planta": 8, "Octava planta": 8, "8ª planta": 8, "8a planta": 8,
    "novena planta": 9, "Novena planta": 9, "9ª planta": 9, "9a planta": 9,
    "décima planta": 10, "Décima planta": 10, "10ª planta": 10, "10a planta": 10,
    "undécima planta": 11, "Undécima planta": 11, "11ª planta": 11, "11a planta": 11,
    "duodécima planta": 12, "Duodécima planta": 12, "12ª planta": 12, "12a planta": 12,
    "decimotercera planta": 13, "Decimotercera planta": 13, "13ª planta": 13, "13a planta": 13,
    "decimocuarta planta": 14, "Decimocuarta planta": 14, "14ª planta": 14, "14a planta": 14,
    "decimoquinta planta": 15, "Decimoquinta planta": 15, "15ª planta": 15, "15a planta": 15,
    "decimosexta planta": 16, "Decimosexta planta": 16, "16ª planta": 16, "16a planta": 16,
    "decimoséptima planta": 17, "Decimoséptima planta": 17, "17ª planta": 17, "17a planta": 17,
    "decimoctava planta": 18, "Decimoctava planta": 18, "18ª planta": 18, "18a planta": 18,
    "decimonovena planta": 19, "Decimonovena planta": 19, "19ª planta": 19, "19a planta": 19,
    "vigésima planta": 20, "Vigésima planta": 20, "20ª planta": 20, "20a planta": 20
}



In [70]:
# Sustituir los valores textuales por numéricos en la columna "planta"
dfrdetalles["planta"] = dfrdetalles["planta"].map(mapeo_plantas).fillna(dfrdetalles["planta"])

C:\Users\extas\AppData\Local\Temp\ipykernel_11376\3088722886.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfrdetalles["planta"] = dfrdetalles["planta"].map(mapeo_plantas).fillna(dfrdetalles["planta"])
C:\Users\extas\AppData\Local\Temp\ipykernel_11376\3088722886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrdetalles["planta"] = dfrdetalles["planta"].map(mapeo_plantas).fillna(dfrdetalles["planta"])


In [71]:
dfrdetalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              2108 non-null   object 
 1   precio              2108 non-null   object 
 2   cantidad_visitas    2108 non-null   object 
 3   fecha_publicacion   2108 non-null   object 
 4   referencia          2108 non-null   object 
 5   m2_constr           2104 non-null   object 
 6   m2_utiles           1505 non-null   object 
 7   dormitorios         2103 non-null   float64
 8   banos               2108 non-null   object 
 9   titularanuncio      1380 non-null   object 
 10  descripcion_larga   2085 non-null   object 
 11  amueblado           4 non-null      object 
 12  alarma              5 non-null      object 
 13  num_pisos           24 non-null     object 
 14  num_plantas         31 non-null     object 
 15  puerta_seguridad    25 non-null     object 
 16  acceso

In [72]:
dfrdetalles.head(1)

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion,ano_construccion
0,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde, Madrid, Madrid",106.500 €,Nº Visitas 5857,02/04/2024,RP2842024125161,64 m²,Metros útiles: 60 m²,2.0,1,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito en el RAIN: 000517/20. En venta este inmueble de 64m² , ubicado en una 5º planta . Consta de 2 amplios dormitorios ( antes 3) con armarios empotrados, salón y cocina independientes, baño con plato de ducha . Además cuenta con aire acondicionado en salón , suelos de tarima .Muy luminoso!. PARA ENTRAR A VIVIR, NO DEJES PASAR ESTA OPORTUNIDAD. San Cristóbal de los Ángeles es un barrio del extrarradio de Madrid, humilde y lleno de vida. Rodeado de grandes zonas verdes, como nuestro precioso parque, conocido por su estanque. Un lugar ideal para pasear, hacer deporte o celebraciones al aire libre. La zona cuenta con comercios muy variados, un amplio mercado y dos supermercados, además de zonas para practicar deporte, como son una cancha de baloncesto, pistas donde poder jugar al tenis y un gran campo de futbol donde los pequeños pueden practicar y aprender su deporte favorito. También cuenta con varios colegios cercanos y un centro de educación secundaria. El barrio tiene muy buena comunicación, dos líneas de autobuses de la EMT (59 Y 79) y Metro ( Linea 3) parada San Cristóbal de los Ángeles, a 15 minutos de Sol. También contamos con parada de Cercanías RENFE, tan solo a 11 minutos de la estación de Atocha y 14 minutos de Sol. Carretera salida a la avenida de Andalucía con salidas próximas a la M30, M40, M45, M50. En el barrio se encuentran situados el Centro de Salud San Cristóbal y el Hospital de día Villaverde. El hospital 12 de Octubre está a diez minutos.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,Tipo de suelo: Tarima normal,NaN,Aire acondicionado,Antigüedad: Más de 50 años,NaN,Agua caliente: Individual,Estado: Perfecto Estado,NaN,NaN,NaN,NaN,Año de construcción: 1962


In [73]:
dfrdetalles["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                          1874
Año de construcción: 1960      23
Año de construcción: 1930      20
Año de construcción: 1965      19
Año de construcción: 1970      14
Año de construcción: 1962      10
Año de construcción: 1940      10
Año de construcción: 1964       9
Año de construcción: 1925       8
Año de construcción: 1972       8
Año de construcción: 1966       8
Año de construcción: 1900       7
Año de construcción: 1920       7
Año de construcción: 1961       7
Año de construcción: 1969       6
Año de construcción: 1959       5
Año de construcción: 1950       5
Año de construcción: 1994       4
Año de construcción: 1968       4
Año de construcción: 1986       3
Año de construcción: 1958       3
Año de construcción: 1955       3
Año de construcción: 1967       3
Año de construcción: 1948       2
Año de construcción: 1974       2
Año de construcción: 2000       2
Año de construcción: 1979       2
Año de construcción: 1945       2
Año de construcción: 1973      

In [74]:
dfrdetalles["antiguedad"].value_counts(dropna=False)

antiguedad
NaN                                1709
Antigüedad: Más de 50 años          312
Antigüedad: Entre 35 y 50 Años       63
Antigüedad: Mas de 25 Años            9
Antigüedad:  Entre 20 y 25 Años       5
Antigüedad: Entre 25 y 35 años        4
Antigüedad: Menos de 5 Años           3
Antigüedad: Entre 10 y  15 Años       2
Antigüedad: Entre 15 y 20 Años        1
Name: count, dtype: int64

In [75]:
dfrdetalles["antiguedad"]=dfrdetalles["antiguedad"].str.strip()
# Diccionario con los patrones de reemplazo para cada columna
replacements = {
    "m2_utiles": ["Metros útiles:","Metros útiles", "m2", "m²", ":"],
    "armarios": ["Armarios",":"],
    "banos": [":", "Baños"],
    "m2_constr": ["m²", ":"],
    "precio": ["€"],
    "orientacion": ["Orientacion:"],
    "calefaccion": ["Calefacción:"],
    "cantidad_visitas": ["Nº Visitas"],
    "tipo_suelo": ["Tipo de suelo:",":"],
    "tipo_fachada": ["Tipo de fachada:"],
    "antiguedad": ["Antigüedad:"],
    "estado": ["Estado:"],
    "agua_caliente": ["Agua caliente:"],
    "gastos_comunidad": ["Comunidad:","€"],
    "orientacion": ["Orientación:"],
    "emisiones_CO2": ["Emisiones: ","Kg CO2/m² año"],
    "consumo_energia": ["Consumo: ","h/m² año"],
    "ano_construccion": ["Año de construcción:"]
}


# Aplicar los reemplazos
for column, patterns in replacements.items():
    for pattern in patterns:
        dfrdetalles[column] = dfrdetalles[column].str.replace(pattern, "", case=False, regex=True)


C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1996363014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrdetalles["antiguedad"]=dfrdetalles["antiguedad"].str.strip()
C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1996363014.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrdetalles[column] = dfrdetalles[column].str.replace(pattern, "", case=False, regex=True)


In [76]:
dfrdetalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              2108 non-null   object 
 1   precio              2108 non-null   object 
 2   cantidad_visitas    2108 non-null   object 
 3   fecha_publicacion   2108 non-null   object 
 4   referencia          2108 non-null   object 
 5   m2_constr           2102 non-null   object 
 6   m2_utiles           1505 non-null   object 
 7   dormitorios         2103 non-null   float64
 8   banos               2100 non-null   object 
 9   titularanuncio      1380 non-null   object 
 10  descripcion_larga   2085 non-null   object 
 11  amueblado           4 non-null      object 
 12  alarma              5 non-null      object 
 13  num_pisos           24 non-null     object 
 14  num_plantas         31 non-null     object 
 15  puerta_seguridad    25 non-null     object 
 16  acceso

In [77]:
dfrdetalles['antiguedad'] = dfrdetalles['antiguedad'].str.strip()

# Completar los valores nulos en la columna 'antiguedad' con los valores de 'ano_construccion'
dfrdetalles['antiguedad'] = dfrdetalles['antiguedad'].fillna(dfrdetalles['ano_construccion'])
dfrdetalles= dfrdetalles.drop(columns=('ano_construccion'))



C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1191007325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrdetalles['antiguedad'] = dfrdetalles['antiguedad'].str.strip()
C:\Users\extas\AppData\Local\Temp\ipykernel_11376\1191007325.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfrdetalles['antiguedad'] = dfrdetalles['antiguedad'].fillna(dfrdetalles['ano_construccion'])


In [78]:
dfrdetalles["antiguedad"].value_counts(dropna=False)

antiguedad
NaN                    1639
Más de 50 años          312
Entre 35 y 50 Años       63
 1930                    10
Mas de 25 Años            9
 1965                     7
 1970                     6
Entre 20 y 25 Años        5
 1940                     4
Entre 25 y 35 años        4
 1960                     4
 1962                     4
 1950                     4
Menos de 5 Años           3
 1925                     2
 1969                     2
 1964                     2
Entre 10 y  15 Años       2
 1972                     2
 1973                     1
 1920                     1
 1977                     1
 2006                     1
 1961                     1
 1974                     1
 1933                     1
 1959                     1
 1968                     1
 1951                     1
 1923                     1
Entre 15 y 20 Años        1
 1995                     1
 45                       1
 1990                     1
 1975                     1
 1954    

In [79]:
dfrdetalles.tail()

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion
2103,"Piso en venta en CALLE ORIO, 4, Los Ángeles, Villaverde, Madrid, Madrid",A consultar,240,22/05/2024,RP332024126817,NaN,NaN,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,Cemento,Baldosa,Gas Natural,NaN,Más de 50 años,NaN,Gas natural,A reformar,NaN,NaN,NaN,Norte
2104,"Piso en venta en CALLE CANCION DEL OLVIDO, 12, Los Ángeles, Villaverde, Madrid, Madrid",A consultar,218,24/06/2024,RP332024127980,90,71,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"65,00",NaN,NaN,Otros materiales,NaN,NaN,NaN,1960,Ascensor,Individual,NaN,NaN,NaN,NaN,NaN
2105,"Piso en venta en CALLE LA ALEGRIA DE LA HUERTA, 17, Los Ángeles, Villaverde, Madrid, Madrid",A consultar,172,15/07/2024,RP332024128702,79,63,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,Eléctrica,Aire acondicionado,NaN,Ascensor,Gas natural,Perfecto Estado,NaN,NaN,NaN,NaN
2106,"Piso en venta en CALLE Ochagavia, Valdezarza, Moncloa-Aravaca, Madrid, Madrid",A consultar,845,11/06/2024,RP402024127484,76,69,3.0,1,"Piso en venta en CALLE Ochagavia, Valdezarza, Moncloa-Aravaca, Madrid, Madrid","Redpiso pone en venta un piso en calle Ochagavia, este es una segunda planta exterior con ascensor, esto hace que su iluminación sea extraordinaria. El inmueble cuenta con tres dormitorios, salón cocina y baño. Uno de los dormitorios tiene un amplio armario empotrado en perfectas condiciones. El salón tiene acceso a su estupenda terraza. El barrio donde se encuentra esta compuesto por múltiples zonas verdes sin alejarte de comercios colegios. Gracias a su magnífica ubicación, se encontrará con diversas paradas de autobuses, las paradas de metro de Valdezarza, y salida directa a la M30. ¡No vas a querer perderte esta gran oportunidad de comprar! Llámanos e infórmate.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,NaN,NaN,NaN,Ascensor,NaN,NaN,36,172 KW,2,Oeste
2107,"Piso en venta en Usera, Madrid, Madrid",A consultar,535,08/05/2024,RP782024126316,104,84,3.0,2,PISO VENDIDO EN UNA SEMANA,"Redpiso vende en exclusiva amplia vivienda en la mejor zona de la Meseta de Orcasitas. Consta de 3 dormitorios, amplio salón, terraza de unos 7 metros, cocina independiente, despensa, baño y aseo. Armarios empotrados amplios en los dormitorios. Calefacción central incluida en la cuota de la comunidad. Finca sin ascensor. Con todo tipo de comercios y servicios cercanos: centros deportivos, centro de mayores, colegios, centro de salud, Mercadona, entidades bancarias.. . Situado en zona tranquila y bien comunicada tanto con líneas EMT (78, 81, 116) y Renfe Orcasitas (línea C5). Fácil aparcamiento en la zona y accesos rápidos a M30 y M40. La vivienda necesita reforma integral, incluida la instalación de calefacción. Inmueble bajo propuesta y en estudio del préstamo hipotecario, no se puede visitar mientras este la operación en marcha.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,NaN,NaN,NaN,NaN,Gas natural,A reformar,53,253 KW,NaN,Sur


In [80]:
# eliminacion de columnas 
#redpiso_detalles= redpiso_detalles.drop(columns=["campo13","tiempo2","campo14","campo15","tiempo3","campo16","campo17","campo18","tiempo4",
#"campo19"])

In [81]:
dfrdetalles.shape

(2108, 38)

In [82]:
# Reemplazo de valores nulos por ceros y los otros por unos 
#columnas_binario= ["amueblado","alarma","conserje","puerta_seguridad","acceso_minusvalido","cocina_equipada","aire_acondicionado",
     #              "ascensor","piscina","trastero","Zonas_verdes"]

#redpiso_detalles[columnas_binario]= redpiso_detalles[columnas_binario].notnull().astype(int)


-----------

In [83]:
redpiso_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   precio                 2108 non-null   object
 1   url_imagen             1385 non-null   object
 2   reservado              2108 non-null   int64 
 3   m2_constr              2101 non-null   object
 4   cantidad_habitaciones  2103 non-null   object
 5   url                    2108 non-null   object
 6   descripcion            2108 non-null   object
 7   contacto               2108 non-null   object
 8   officelink_url         2108 non-null   object
 9   nombre_oficina         2108 non-null   object
 10  referencia             2108 non-null   object
dtypes: int64(1), object(10)
memory usage: 181.3+ KB


In [84]:
dfrdetalles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              2108 non-null   object 
 1   precio              2108 non-null   object 
 2   cantidad_visitas    2108 non-null   object 
 3   fecha_publicacion   2108 non-null   object 
 4   referencia          2108 non-null   object 
 5   m2_constr           2102 non-null   object 
 6   m2_utiles           1505 non-null   object 
 7   dormitorios         2103 non-null   float64
 8   banos               2100 non-null   object 
 9   titularanuncio      1380 non-null   object 
 10  descripcion_larga   2085 non-null   object 
 11  amueblado           4 non-null      object 
 12  alarma              5 non-null      object 
 13  num_pisos           24 non-null     object 
 14  num_plantas         31 non-null     object 
 15  puerta_seguridad    25 non-null     object 
 16  acceso

In [85]:
redpiso_normal["referencia"].value_counts(dropna=False)

referencia
RP302023121517     2
RP2842024125161    1
RP462023121637     1
RP082023121205     1
RP022022105196     1
                  ..
RP2062024131501    1
RP1892024128590    1
RP182024130223     1
RP182024130127     1
RP782024126316     1
Name: count, Length: 2107, dtype: int64

In [86]:
dfrdetalles["referencia"].value_counts(dropna=False)

referencia
RP302023121517     2
RP2842024125161    1
RP462023121637     1
RP082023121205     1
RP022022105196     1
                  ..
RP2062024131501    1
RP1892024128590    1
RP182024130223     1
RP182024130127     1
RP782024126316     1
Name: count, Length: 2107, dtype: int64

In [87]:
redpiso_normal.query("referencia=='RP302023121517'")

,precio,url_imagen,reservado,m2_constr,cantidad_habitaciones,url,descripcion,contacto,officelink_url,nombre_oficina,referencia
899,300.000 €,https://www.redpiso.es/upload/properties/thumbs/img_1890850_min.jpg,0,172,2,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pinzon-vista-alegre-carabanchel-madrid-madrid-RP302023121517,"Piso en venta en CALLE PINZON, Vista Alegre, Carabanchel, Madrid, Madrid",911921002,https://www.redpiso.es/oficina/inmobiliaria-carabanchel-vistalegre-carpetana/venta,REDPISO CARABANCHEL VISTALEGRE-CARPETANA,RP302023121517
900,300.000 €,NaN,0,172,2,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pinzon-vista-alegre-carabanchel-madrid-madrid-RP302023121517,"Piso en venta en CALLE PINZON, Vista Alegre, Carabanchel, Madrid, Madrid",911921002,https://www.redpiso.es/oficina/inmobiliaria-carabanchel-vistalegre-carpetana/venta,REDPISO CARABANCHEL VISTALEGRE-CARPETANA,RP302023121517


In [88]:
dfrdetalles.query("referencia=='RP302023121517'")

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion
899,"Piso en venta en CALLE PINZON, Vista Alegre, Carabanchel, Madrid, Madrid",300.000,835,07/12/2023,RP302023121517,172,102,2.0,2,"Encanto, Comodidad y tu nuevo hogar a un paso del metro","¡Bienvenido al hogar de tus sueños en Vistalegre! Este encantador PISO EN PLANTA BAJA CON PATIO DE 70 M2 EN PROPIEDAD te está esperando con los brazos abiertos. Con 110 metros cuadrados de pura comodidad, es perfecto para compradores exigentes y familias que buscan un espacio acogedor.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,Gas Natural,NaN,Entre 20 y 25 Años,Ascensor,NaN,NaN,NaN,NaN,NaN,Este
900,"Piso en venta en CALLE PINZON, Vista Alegre, Carabanchel, Madrid, Madrid",300.000,836,07/12/2023,RP302023121517,172,102,2.0,2,"Encanto, Comodidad y tu nuevo hogar a un paso del metro","¡Bienvenido al hogar de tus sueños en Vistalegre! Este encantador PISO EN PLANTA BAJA CON PATIO DE 70 M2 EN PROPIEDAD te está esperando con los brazos abiertos. Con 110 metros cuadrados de pura comodidad, es perfecto para compradores exigentes y familias que buscan un espacio acogedor.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,Gas Natural,NaN,Entre 20 y 25 Años,Ascensor,NaN,NaN,NaN,NaN,NaN,Este


In [89]:
dfrdetalles.query("referencia=='RP2332024131076'")

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion
2097,"Piso en venta en CALLE INFANTA MERCEDES, 11, Cuatro Caminos, Tetuán, Madrid, Madrid",420.000,72,08/10/2024,RP2332024131076,61,49,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,NaN,Aire acondicionado,NaN,Ascensor,Central,Perfecto Estado,NaN,NaN,1,NaN


In [90]:
dfrdetalles.shape,redpiso_normal.shape

((2108, 38), (2108, 11))

In [91]:
# Eliminar las filas duplicadas basándote en la columna "referencia" y quedarte con la primees= dfrdetalles.drop_duplicates(subset="referencia", keep="first")
dfrdetalles= dfrdetalles.drop_duplicates(subset="referencia", keep="first")
redpiso_normal=redpiso_normal.drop_duplicates(subset="referencia", keep="first")

In [92]:
# Asignar las columnas de redpiso_normal a dfr_normal
dfr_normal = redpiso_normal[["url", "nombre_oficina", "referencia"]]

# Realizar la combinación (merge) de los DataFrames
df_merge = pd.merge(dfrdetalles, dfr_normal, how="inner", on="referencia", indicator=True)


In [93]:
df_merge.query("referencia=='RP2332024131076'")

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion,url,nombre_oficina,_merge
2096,"Piso en venta en CALLE INFANTA MERCEDES, 11, Cuatro Caminos, Tetuán, Madrid, Madrid",420.000,72,08/10/2024,RP2332024131076,61,49,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,NaN,Aire acondicionado,NaN,Ascensor,Central,Perfecto Estado,NaN,NaN,1,NaN,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-infanta-mercedes-11-cuatro-caminos-tetuan-madrid-madrid-RP2332024131076,REDPISO CUATRO CAMINOS-BERNABÉU,both


In [94]:
df_merge.shape

(2107, 41)

In [95]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107 entries, 0 to 2106
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   titulo              2107 non-null   object  
 1   precio              2107 non-null   object  
 2   cantidad_visitas    2107 non-null   object  
 3   fecha_publicacion   2107 non-null   object  
 4   referencia          2107 non-null   object  
 5   m2_constr           2101 non-null   object  
 6   m2_utiles           1504 non-null   object  
 7   dormitorios         2102 non-null   float64 
 8   banos               2099 non-null   object  
 9   titularanuncio      1379 non-null   object  
 10  descripcion_larga   2084 non-null   object  
 11  amueblado           4 non-null      object  
 12  alarma              5 non-null      object  
 13  num_pisos           24 non-null     object  
 14  num_plantas         31 non-null     object  
 15  puerta_seguridad    25 non-null     ob

In [96]:
# Filtrar filas donde el indicador _merge no es 'both'
df_merge[df_merge['_merge'] != 'both']


,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion,url,nombre_oficina,_merge


In [97]:
# Eliminar las filas donde el valor de la columna 'precio' es "A consultar"
df_merge= df_merge[df_merge['precio'] != "A consultar"]


In [98]:
url_especifica = 'https://www.redpiso.es/inmueble/piso-en-venta-en-hispanoamerica-chamartin-madrid-madrid-RP902024129170'
df_merge.loc[df_merge['url'].str.contains(url_especifica), ['m2_constr', 'planta']] = [155, 1]
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/en-venta-en-calle-la-alegria-de-la-huerta-los-angeles-villaverde-madrid-madrid-RP332024131222'), ['m2_constr','tiene_piscina','nuda_propiedad']] = [90,0,1]
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP352023120094'), ['m2_constr','tiene_piscina','terraza','trastero']] = [75,0,1,1]


In [99]:
df_merge['precio'] = df_merge['precio'].apply(lambda x: str(x).replace('.', '') if isinstance(x, str) and float(x.replace('.', '')) >= 1000000 else x)
col_tiponuevo= ["precio","m2_constr"]

df_merge[col_tiponuevo] = df_merge[col_tiponuevo].astype("float")

redpiso_detalles2["dormitorios"]= redpiso_detalles2["dormitorios"].astype(str)
redpiso_detalles2["banos"]= redpiso_detalles2["banos"].astype(str)
redpiso_detalles2["espacio_metros_cuadrados"]= redpiso_detalles2["espacio_metros_cuadrados"].astype(str) 
redpiso_detalles2["cantidad_armarios"]= redpiso_detalles2["cantidad_armarios"].astype(str)
redpiso_detalles2["cantidad_visitas"]= redpiso_detalles2["cantidad_visitas"].astype(str)

In [100]:
df_merge["precio"].isna().sum()

0

In [101]:
df_merge.shape

(1467, 42)

In [102]:
df_merge.duplicated().sum()

0

In [103]:
df_merge["orientacion"].value_counts(dropna=False)

orientacion
NaN          715
 Sur         162
 Este        151
 Oeste       125
 Norte        80
 Sureste      71
 Suroeste     59
 Noroeste     53
 Noreste      51
Name: count, dtype: int64

In [104]:
df_merge.isna().sum()

titulo                   0
precio                   0
cantidad_visitas         0
fecha_publicacion        0
referencia               0
m2_constr                0
m2_utiles              382
dormitorios              2
banos                    4
titularanuncio         498
descripcion_larga       11
amueblado             1466
alarma                1464
num_pisos             1451
num_plantas           1447
puerta_seguridad      1449
acceso_minusvalido    1454
porteria              1466
terraza               1466
zonas_verdes          1336
planta                 976
garaje                1160
trastero              1210
gastos_comunidad      1436
tiene_piscina         1427
exterior               342
tipo_fachada          1286
tipo_suelo             970
calefaccion            890
aire_acondicionado    1035
antiguedad            1139
ascensor               564
agua_caliente          775
estado                 215
emisiones_CO2         1206
consumo_energia       1207
armarios               881
o

In [105]:
df_merge["web"]="redpiso"
df_merge["fecha_descarga"]="28/10/2024"

In [106]:
df_merge.isna().sum()

titulo                   0
precio                   0
cantidad_visitas         0
fecha_publicacion        0
referencia               0
m2_constr                0
m2_utiles              382
dormitorios              2
banos                    4
titularanuncio         498
descripcion_larga       11
amueblado             1466
alarma                1464
num_pisos             1451
num_plantas           1447
puerta_seguridad      1449
acceso_minusvalido    1454
porteria              1466
terraza               1466
zonas_verdes          1336
planta                 976
garaje                1160
trastero              1210
gastos_comunidad      1436
tiene_piscina         1427
exterior               342
tipo_fachada          1286
tipo_suelo             970
calefaccion            890
aire_acondicionado    1035
antiguedad            1139
ascensor               564
agua_caliente          775
estado                 215
emisiones_CO2         1206
consumo_energia       1207
armarios               881
o

In [107]:
df_merge.query("descripcion_larga.isnull()")[["url","titulo","precio","estado","terraza"]]

,url,titulo,precio,estado,terraza
300,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-patio-de-santiago-nueva-numancia-puente-de-vallecas-madrid-madrid-RP022024127656,"Piso en venta en CALLE PATIO DE SANTIAGO, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",185.0,Perfecto Estado,NaN
326,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-brigida-alonso-nueva-numancia-puente-de-vallecas-madrid-madrid-RP022024130100,"Piso en venta en CALLE BRIGIDA ALONSO, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",199.0,Perfecto Estado,NaN
1087,https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-ascension-bielsa-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202299726,"Apartamento en venta en CALLE ASCENSION BIELSA, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",95.0,Perfecto Estado,NaN
1114,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-almonacid-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202192852,"Piso en venta en CALLE ALMONACID, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",112.0,NaN,NaN
1136,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-ascension-bielsa-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202299728,"Ático en venta en CALLE ASCENSION BIELSA, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",125.0,Perfecto Estado,NaN
1244,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-lopez-grass-nueva-numancia-puente-de-vallecas-madrid-madrid-RP022023112335,"Ático en venta en CALLE LOPEZ GRASS, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",160.0,Perfecto Estado,NaN
1262,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-ramon-perez-de-ayala-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202189311,"Piso en venta en CALLE Ramón Pérez DE AYALA, Nueva Numancia, Puente de Vallecas, Madrid, Madrid",169.0,Perfecto Estado,NaN
1287,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-camarena-aluche-latina-madrid-madrid-RP272024123215,"Piso en venta en CALLE CAMARENA, Aluche, Latina, Madrid, Madrid",175.0,Perfecto Estado,NaN
1302,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-valmojado-aluche-latina-madrid-madrid-RP272024123901,"Piso en venta en CALLE VALMOJADO, Aluche, Latina, Madrid, Madrid",180.0,Perfecto Estado,NaN
1568,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fortunata-y-jacinta-cuatro-caminos-tetuan-madrid-madrid-RP2332024127264,"Piso en venta en CALLE FORTUNATA Y JACINTA, Cuatro Caminos, Tetuán, Madrid, Madrid",400.0,A reformar,NaN


In [108]:
d="Metros: 81 m² Metros útiles: 75 m² Dormitorios: 3 Baños: 2 Armarios: 2 Estado: A reformar Orientación: Este Exterior Ascensor Cocina equipada Zonas verdes Aire acondicionado Agua caliente: Gas natural Calefacción: Individual Tipo de suelo: Parquet Antigüedad: Más de 50 años Año de construcción: 1950 Comunidad: 80,00 € Acceso minusválidos Plantas: 4 Número de terrazas: 1"
#df_merge["descripcion_entendida"]
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fortunata-y-jacinta-cuatro-caminos-tetuan-madrid-madrid-RP2332024127264'), ['descripcion_larga','tiene_piscina','terraza','trastero']] = [d,0,1,0]
d= "Metros: 78 m² Metros útiles: 68 m² Dormitorios: 2 Baños: 1 Cal. energética: Calificación energética f  Estado: Perfecto Estado Orientación: Este Exterior Trastero Cocina equipada Agua caliente: Individual Calefacción: Individual Tipo de suelo: Tarima normal Antigüedad: Entre 35 y 50 Años Comunidad: 30,00 € Número de terrazas: 1"
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-calle-valmojado-aluche-latina-madrid-madrid-RP272024123901'), ['descripcion_larga','tiene_piscina','terraza','trastero']] = [d,0,1,1]
d="Metros: 62 m² Metros útiles: 60 m² Dormitorios: 3 Baños: 1 Cal. energética: Calificación energética f  Estado: Perfecto Estado Orientación: Noroeste Exterior Ascensor Cocina equipada Zonas verdes Agua caliente: Individual Calefacción: Individual Tipo de suelo: Tarima Flotante Antigüedad: Entre 35 y 50 Años Año de construcción: 1965 Acceso minusválidos Número de terrazas: 1"
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-calle-camarena-aluche-latina-madrid-madrid-RP272024123215'), ['descripcion_larga','tiene_piscina','terraza','ascensor']] = [d,0,1,1]
d="Metros: 43 m² Dormitorios: 1 Baños: 1 Cal. energética: Calificación energética e  Consumo: 268 KW h/m² año Emisiones: 56 Kg CO2/m² año Estado: Perfecto Estado Exterior Ascensor Cocina equipada Aire acondicionado Agua caliente: Individual Calefacción: Eléctrica Tipo de suelo: Tarima Flotante Antigüedad: Entre 25 y 35 años Año de construcción: 1992 Puerta de seguridad Número de terrazas: 1"
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/atico-en-venta-en-calle-lopez-grass-nueva-numancia-puente-de-vallecas-madrid-madrid-RP022023112335'), ['descripcion_larga','tiene_piscina','terraza']] = [d,0,1]
d=" Metros: 56 m² Metros útiles: 28 m² Dormitorios: 1 Baños: 1 Cal. energética: Calificación energética e  Consumo: 262 KW h/m² año Emisiones: 45 Kg CO2/m² año Estado: Perfecto Estado Exterior Ascensor Trastero Aire acondicionado Agua caliente: Individual Calefacción: Individual Tipo de suelo: Gres Antigüedad: Entre 15 y 20 Años"
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-ascension-bielsa-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202299726'), ['descripcion_larga','tiene_piscina','terraza']] = [d,0,0]
d="Metros: 85 m² Dormitorios: 4 Baños: 1 Cal. energética: Calificación energética e  Consumo: 219 KW h/m² año Emisiones: 45 Kg CO2/m² año Estado: Perfecto Estado Exterior Aire acondicionado Agua caliente: Individual Calefacción: Eléctrica Tipo de suelo: Parquet Tipo de fachada: Ladrillo Antigüedad: Mas de 25 Años Año de construcción: 1979 Puerta de seguridad Número de terrazas: 2"
df_merge.loc[df_merge['url'].str.contains('https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-ascension-bielsa-nueva-numancia-puente-de-vallecas-madrid-madrid-RP02202299726'), ['descripcion_larga','tiene_piscina','terraza']] = [d,0,1]

In [109]:
pd.set_option('display.max_colwidth', None)

In [110]:
redpiso_unido= df_merge

In [111]:
redpiso_unido["nombre_oficina"]=redpiso_unido["nombre_oficina"].str.replace("REDPISO", "", regex=False).str.strip()
redpiso_unido['tipologia'] = redpiso_unido['titulo'].str.split().str[0]
redpiso_unido["calle"] = redpiso_unido["titulo"].apply(extraer_calle)
redpiso_unido["titulo"] = redpiso_unido["titulo"].str.replace(", Madrid, Madrid", "", regex=False)


In [112]:
# Asignar valor a 'calle' basándose en la condición de la URL
redpiso_unido.loc[redpiso_unido['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP622024129118'), ['calle']] = ['Calle de Badalona']
redpiso_unido.loc[redpiso_unido['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP202024123803'), ['calle']] = ['Calle del Congosto']


In [113]:
u='https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP832024126556'
redpiso_unido.loc[redpiso_unido['url'].str.contains(u), ['distrito','cod_distrito','barrio','cod_barrio','codigo_Postal']] = ['Usera',12,'Moscardó',125,'28026']
u='https://www.redpiso.es/inmueble/casa-en-venta-en-madrid-madrid-RP622024130473'
redpiso_unido.loc[redpiso_unido['url'].str.contains(u), ['distrito','cod_distrito','barrio','cod_barrio','codigo_Postal']] = ['Fuencarral-El Pardo',8,'Valverde',86,'28034']

In [114]:
#redpiso_unido[["barrio", "distrito"]] = redpiso_unido["titulo"].apply(lambda x: pd.Series(extraer_barrio_distrito(x)))
#redpiso_unido[["titulo","tipologia","calle","distrito","barrio"]]

In [115]:
# Creacion de nuevas columnas donde metere los valores de de distrito, barrio, codigos postal y el codigo por distrito y barrio.

nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
redpiso_unido[nuevas_columnas]=np.nan




In [116]:
# tengo en algunos casos el barrio, lo busco en el dataframe dbcp_ordenado si lo encuentro relleno el resto de los campos , si no lo encuento pongo el barrio en la zolumna zona y busco el distrito para poner 
# si lo encuentro el codigo del distrito

#### **_Agregacion de barrios y distritos con sus codigos respectivos/ Redpiso._**

In [117]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [118]:
zona_barrio_dict = {
    'Sanchinarro': {     
        'barrio': 'Valdefuentes'
    },
    'San Andrés': {       
        'barrio': 'Villaverde Alto'
    },
    'Virgen del Cortijo': {      
        'barrio': 'Valdefuentes'
    },
    'Montecarmelo': {       
        'barrio': 'Mirasierra'
    }
    ,
    'Villaverde Bajo': {       
        'barrio': 'Los Rosales'
    },
    'villaverdebajo': {       
        'barrio': 'Los Rosales'
    }
}


# Cambiar el tipo de dato de las columnas "barrio" y "distrito" a object
redpiso_unido["barrio"] = redpiso_unido["barrio"].astype(object)
redpiso_unido["distrito"] = redpiso_unido["distrito"].astype(object)
redpiso_unido["cod_barrio"] = redpiso_unido["cod_barrio"].astype(object)
redpiso_unido["cod_distrito"] = redpiso_unido["cod_distrito"].astype(object)
redpiso_unido["codigo_Postal"] = redpiso_unido["codigo_Postal"].astype(object)
dbcp_ordenado.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dbcp_ordenado.loc[:, 'distrito']]
dbcp_ordenado.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dbcp_ordenado.loc[:, 'barrio']]

def clasificar_distrito_y_barrio_por_columnas(df, dbcp_ordenado):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Recorrer las columnas en orden de prioridad: titulo, nombre_oficina
        for columna in ['titulo', 'nombre_oficina']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un barrio
                barrio_encontrado = next((barrio for barrio in dbcp_ordenado["barrio_n"] if barrio in valor_normalizado), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = dbcp_ordenado.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en dbcp_ordenado
                    result = dbcp_ordenado.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal
                # Verificamos si el valor normalizado contiene el nombre de un distrito
                distrito_encontrado = next((distrito for distrito in dbcp_ordenado["distrito_n"] if distrito in valor_normalizado), None)
                if distrito_encontrado and pd.isna(fila["distrito"]):
                    df.at[idx, "distrito"] = dbcp_ordenado.query(f"distrito_n == '{distrito_encontrado}'")["distrito"].values[0]
                    result = dbcp_ordenado.query(f"distrito_n == '{distrito_encontrado}'")                    
                    cod_distrito = result["cod_distrito"].values[0]
                    df.at[idx, "cod_distrito"] = cod_distrito
            else:
                #zona_encontrado = next((barrio for barrio in dbcp_ordenado["barrio_n"] if barrio in valor_normalizado), None)
                barrio_encontrado = next((zona_barrio_dict[zona]["barrio"] for zona in zona_barrio_dict if zona_barrio_dict[zona]["barrio"] in dbcp_ordenado["barrio_n"]), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = dbcp_ordenado.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en dbcp_ordenado
                    result = dbcp_ordenado.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal

In [119]:


dfcalles.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del distrito']]
dfcalles.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del barrio']]

# Normalizar y limpiar la columna 'Nombre de la vía'
dfcalles['calle_n'] = dfcalles['Nombre de la vía'].astype(str).str.strip().str.replace("Calle", "").str.replace("Avenida", "").str.replace("Plaza", "").str.strip().apply(normalize_string).str.replace(" ", "")

def clasificar_distrito_y_barrio_calle(df, dfcalles):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Verificar si la columna "barrio" es nula
        if pd.isna(fila["barrio"]):
            # Recorrer las columnas en orden de prioridad: calle
            for columna in ['calle']:
                valor = fila[columna]
                
                # Verificamos que el valor no sea nulo
                if pd.notna(valor):
                    valor_normalizado = normalize_string(valor.strip().replace("Calle", "").replace("Avenida", "").replace("Plaza", "").replace(" ", ""))
                    
                    # Verificamos si el valor normalizado contiene el nombre de una calle
                    calle_encontrada = next((calle for calle in dfcalles["calle_n"] if calle in valor_normalizado), None)
                    
                    if calle_encontrada:
                        # Buscar los valores cod_barrio, distrito y cod_distrito en dfcalles
                        result = dfcalles.query(f"calle_n == '{calle_encontrada}'")
                        if not result.empty:
                            barrio = result["Nombre del barrio"].values[0]
                            cod_distrito = result["Codigo de distrito"].values[0]
                            cod_barrio = result["Codigo de barrio"].values[0]
                            codigo_Postal = result["Codigo postal"].values[0]
                            distrito_result = result["Nombre del distrito"].values[0]
                            
                            # Asignar los valores encontrados a las columnas correspondientes
                            df.at[idx, "barrio"] = barrio
                            df.at[idx, "cod_barrio"] = cod_barrio
                            df.at[idx, "cod_distrito"] = cod_distrito
                            df.at[idx, "distrito"] = distrito_result
                            df.at[idx, "codigo_Postal"] = codigo_Postal
                        
    return df




In [120]:
# lamar a la función pasándole el dataframe y el dataframe de referencia

clasificar_distrito_y_barrio_por_columnas(redpiso_unido, dbcp_ordenado)

In [121]:
dfcalles.tail()

,Codigo de numero,Codigo de via,Clase de la via,Partícula de la vía,Nombre de la vía,Literal de numeracion,Codigo de distrito,Nombre del distrito,Codigo de barrio,Nombre del barrio,Seccion censal,Codigo postal,Seccion de carteria,Zona Servicio Estacionamiento Regulado,Categoria fiscal,Direccion completa para el numero,Coordenada X (Guia Urbana) cm,Coordenada Y (Guia Urbana) cm,Longitud en S R ETRS89 WGS84,Latitud en S R ETRS89 WGS84,Tipo de la via a la que pertenece el numero,Situacion de la via respecto al terreno,Tipo de denominacion de la via,Parcela catastral del numero,Tipologia del numero,Zona de valor del numero,distrito_n,barrio_n,calle_n
214203,20045135,725,PASEO,DEL,ZURRON,NUM000049,21,BARAJAS,4,TIMON,30,28042,ND,0,5,"PASEO ZURRÓN, 000049",45051964,448025888,3º35'5.92'' W,40º28'10.77'' N,Vía,Nivel,Pleno,0403807VK5800C,Frente fachada,R23T,barajas,timon,zurron
214204,20045138,725,PASEO,DEL,ZURRON,NUM000051,21,BARAJAS,4,TIMON,26,28042,ND,0,5,"PASEO ZURRÓN, 000051",45058080,448025960,3º35'3.33'' W,40º28'10.78'' N,Vía,Nivel,Pleno,0602301VK5800D,Frente fachada,R23T,barajas,timon,zurron
214205,20045141,725,PASEO,DEL,ZURRON,NUM000053,21,BARAJAS,4,TIMON,26,28042,ND,0,5,"PASEO ZURRÓN, 000053",45061025,448027575,3º35'2.11'' W,40º28'11.34'' N,Vía,Nivel,Pleno,0602301VK5800D,Portal,R23T,barajas,timon,zurron
214206,20045144,725,PASEO,DEL,ZURRON,NUM000055,21,BARAJAS,4,TIMON,26,28042,ND,0,5,"PASEO ZURRÓN, 000055",45063500,448027080,3º35'1.04'' W,40º28'11.18'' N,Vía,Nivel,Pleno,0602302VK5800D,Frente fachada,R23T,barajas,timon,zurron
214207,20045147,725,PASEO,DEL,ZURRON,NUM000057,21,BARAJAS,4,TIMON,26,28042,ND,0,5,"PASEO ZURRÓN, 000057",45065371,448026961,3º35'0.24'' W,40º28'11.16'' N,Vía,Nivel,Pleno,0602302VK5800D,Frente fachada,R23T,barajas,timon,zurron


In [122]:
redpiso_unido.query("barrio.notnull()")[["url","titulo","nombre_oficina","cod_barrio","barrio","cod_distrito","distrito","codigo_Postal","calle"]]

,url,titulo,nombre_oficina,cod_barrio,barrio,cod_distrito,distrito,codigo_Postal,calle
0,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-benimamet-san-cristobal-villaverde-madrid-madrid-RP2842024125161,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde",SAN CRISTOBAL,172,San Cristóbal,17,Villaverde,28021,CALLE BENIMAMET
1,https://www.redpiso.es/inmueble/piso-en-venta-en-usera-madrid-madrid-RP782024130863,Piso en venta en Usera,USERA-ORCASITAS,121,Orcasitas,12,Usera,28041,None
2,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,"Piso en venta en Buenavista, Carabanchel",PAU CARABANCHEL,116,Buenavista,11,Carabanchel,28025,None
3,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,"Piso en venta en Buenavista, Carabanchel",PAU CARABANCHEL,116,Buenavista,11,Carabanchel,28025,None
4,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-beniferri-san-cristobal-villaverde-madrid-madrid-RP2842024130186,"Piso en venta en CALLE BENIFERRI, San Cristóbal, Villaverde",SAN CRISTOBAL,172,San Cristóbal,17,Villaverde,28021,CALLE BENIFERRI
...,...,...,...,...,...,...,...,...,...
2088,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-lista-salamanca-madrid-madrid-RP1642024127462,"Piso en venta en CALLE GENERAL PARDIÑAS, Lista, Salamanca",BARRIO DE SALAMANCA,45,Lista,4,Salamanca,28006,CALLE GENERAL PARDIÑAS
2090,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fernandez-de-la-hoz-almagro-chamberi-madrid-madrid-RP1922024124579,"Piso en venta en CALLE FERNÁNDEZ DE LA HOZ, Almagro, Chamberí",CHAMBERÍ - ALMAGRO,74,Almagro,7,Chamberí,28010,CALLE FERNÁNDEZ DE LA HOZ
2094,https://www.redpiso.es/inmueble/piso-en-venta-en-aluche-latina-madrid-madrid-RP272023121667,"Piso en venta en Aluche, Latina",ALUCHE,104,Aluche,10,Latina,28047,None
2095,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fondon-apostol-santiago-hortaleza-madrid-madrid-RP2532022104355,"Piso en venta en CALLE FONDON, Apóstol Santiago, Hortaleza",MANOTERAS,165,Apostol Santiago,16,Hortaleza,28033,CALLE FONDON


In [123]:
# lamar a la función pasándole el dataframe y el dataframe de referencia

clasificar_distrito_y_barrio_calle(redpiso_unido, dfcalles)

,titulo,precio,cantidad_visitas,fecha_publicacion,referencia,m2_constr,m2_utiles,dormitorios,banos,titularanuncio,descripcion_larga,amueblado,alarma,num_pisos,num_plantas,puerta_seguridad,acceso_minusvalido,porteria,terraza,zonas_verdes,planta,garaje,trastero,gastos_comunidad,tiene_piscina,exterior,tipo_fachada,tipo_suelo,calefaccion,aire_acondicionado,antiguedad,ascensor,agua_caliente,estado,emisiones_CO2,consumo_energia,armarios,orientacion,url,nombre_oficina,_merge,nuda_propiedad,web,fecha_descarga,tipologia,calle,distrito,cod_distrito,barrio,cod_barrio,codigo_Postal
0,"Piso en venta en CALLE BENIMAMET, San Cristóbal, Villaverde",106.5,5857,02/04/2024,RP2842024125161,64.0,60,2.0,1,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito en el RAIN: 000517/20. En venta este inmueble de 64m² , ubicado en una 5º planta . Consta de 2 amplios dormitorios ( antes 3) con armarios empotrados, salón y cocina independientes, baño con plato de ducha . Además cuenta con aire acondicionado en salón , suelos de tarima .Muy luminoso!. PARA ENTRAR A VIVIR, NO DEJES PASAR ESTA OPORTUNIDAD. San Cristóbal de los Ángeles es un barrio del extrarradio de Madrid, humilde y lleno de vida. Rodeado de grandes zonas verdes, como nuestro precioso parque, conocido por su estanque. Un lugar ideal para pasear, hacer deporte o celebraciones al aire libre. La zona cuenta con comercios muy variados, un amplio mercado y dos supermercados, además de zonas para practicar deporte, como son una cancha de baloncesto, pistas donde poder jugar al tenis y un gran campo de futbol donde los pequeños pueden practicar y aprender su deporte favorito. También cuenta con varios colegios cercanos y un centro de educación secundaria. El barrio tiene muy buena comunicación, dos líneas de autobuses de la EMT (59 Y 79) y Metro ( Linea 3) parada San Cristóbal de los Ángeles, a 15 minutos de Sol. También contamos con parada de Cercanías RENFE, tan solo a 11 minutos de la estación de Atocha y 14 minutos de Sol. Carretera salida a la avenida de Andalucía con salidas próximas a la M30, M40, M45, M50. En el barrio se encuentran situados el Centro de Salud San Cristóbal y el Hospital de día Villaverde. El hospital 12 de Octubre está a diez minutos.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,Tarima normal,NaN,Aire acondicionado,Más de 50 años,NaN,Individual,Perfecto Estado,NaN,NaN,NaN,NaN,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-benimamet-san-cristobal-villaverde-madrid-madrid-RP2842024125161,SAN CRISTOBAL,both,NaN,redpiso,28/10/2024,Piso,CALLE BENIMAMET,Villaverde,17,San Cristóbal,172,28021
1,Piso en venta en Usera,119.0,714,02/10/2024,RP782024130863,30.0,27,1.0,1,ESTUDIO JARDINES DE YBARRA II,"Redpiso vende en exclusiva ACOGEDOR ESTUDIO en Jardines de Ybarra II, en el Poblado de Orcasitas. Consta de salón (dormitorio), cocina americana y baño con plato de ducha. Calefacción individual eléctrica, agua caliente por termo. Situado en zona muy tranquila y consolidada con todo tipo de comercios y servicios cercanos (Mercadona, centro de salud, instalaciones deportivas, colegios...). Comunicado con varias líneas EMT, Renfe de Orcasitas y accesos rápidos a A42 y M40. Fácil aparcamiento en la zona. Visítalo sin compromiso.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exterior,NaN,NaN,NaN,NaN,NaN,NaN,Individual,Perfecto Estado,NaN,NaN,1,Sur,https://www.redpiso.es/inmueble/piso-en-venta-en-usera-madrid-madrid-RP782024130863,USERA-ORCASITAS,both,NaN,redpiso,28/10/2024,Piso,None,Usera,12,Orcasitas,121,28041
2,"Piso en venta en Buenavista, Carabanchel",120.0,3387,26/04/2024,RP442024126007,37.0,34,1.0,1,¡Tu nuevo hogar te espera en Buena Vista!,"Redpiso vende en exclusiva este encantador estudio a estrenar. Perfecto para aquellos que buscan comodidad, estilo y funcionalidad en un solo espacio.\nUbicado en planta bajo con orientación sur. Cuenta con una cocina americana y un baño moderno equipado con una amplia ducha.

In [124]:
redpiso_unido.query("barrio.isnull()")[["url","titulo","nombre_oficina","cod_barrio","barrio","cod_distrito","distrito","codigo_Postal","calle"]]

,url,titulo,nombre_oficina,cod_barrio,barrio,cod_distrito,distrito,codigo_Postal,calle
22,https://www.redpiso.es/inmueble/piso-en-venta-en-carabanchel-madrid-madrid-RP2492024130821,Piso en venta en Carabanchel,CARABANCHEL ALTO,NaN,NaN,11,Carabanchel,NaN,None
29,https://www.redpiso.es/inmueble/piso-en-venta-en-carabanchel-madrid-madrid-RP2492024127670,Piso en venta en Carabanchel,CARABANCHEL ALTO,NaN,NaN,11,Carabanchel,NaN,None
36,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024127521,Piso en venta en Villaverde,VILLAVERDE BAJO,NaN,NaN,17,Villaverde,NaN,None
38,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024124440,Piso en venta en Villaverde,VILLAVERDE BAJO,NaN,NaN,17,Villaverde,NaN,None
83,https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP622024129005,"Piso en venta en Madrid, Madrid",FUENCARRAL,NaN,NaN,NaN,NaN,NaN,None
104,https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP622024129984,"Piso en venta en Madrid, Madrid",FUENCARRAL,NaN,NaN,NaN,NaN,NaN,None
173,https://www.redpiso.es/inmueble/piso-en-venta-en-carabanchel-madrid-madrid-RP422024130964,Piso en venta en Carabanchel,CARABANCHEL-OPORTO,NaN,NaN,11,Carabanchel,NaN,None
194,https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP832024126556,"Piso en venta en Madrid, Madrid",PLAZA ELIPTICA,NaN,NaN,NaN,NaN,NaN,None
272,https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP422024128975,"Piso en venta en Madrid, Madrid",CARABANCHEL-OPORTO,NaN,NaN,11,Carabanchel,NaN,None
431,https://www.redpiso.es/inmueble/piso-en-venta-en-carabanchel-madrid-madrid-RP2892024122900,Piso en venta en Carabanchel,CARABANCHEL EUGENIA DE MONTIJO,NaN,NaN,11,Carabanchel,NaN,None


In [125]:


# Definir los valores a asignar
valores = {
    'barrio': 'Los Rosales',
    'distrito': 'Villaverde',
    'cod_distrito': 17,
    'cod_barrio': 175,
    'codigo_Postal': 28021
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'VILLAVERDE BAJO') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())

# Definir los valores a asignar
valores = {
    'barrio': 'Ibiza',
    'distrito': 'Retiro',
    'cod_distrito': 3,
    'cod_barrio': 34,
    'codigo_Postal': 28009
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'RETIRO') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())
valores = {
    'barrio': 'Abrantes',
    'distrito': 'Carabanchel',
    'cod_distrito': 17,
    'cod_barrio': 117,
    'codigo_Postal': 28041
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'CARABANCHEL-OPORTO') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())

valores = {
    'barrio': 'Puerta Bonita',
    'distrito': 'Carabanchel',
    'cod_distrito': 11,
    'cod_barrio': 115,
    'codigo_Postal': 28025
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'CARABANCHEL EUGENIA DE MONTIJO') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())

valores = {
    'barrio': 'Buenavista',
    'distrito': 'Carabanchel',
    'cod_distrito': 11,
    'cod_barrio': 116,
    'codigo_Postal': 28025
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'CARABANCHEL ALTO') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())

valores = {
    'barrio': 'San Isidro',
    'distrito': 'Carabanchel',
    'cod_distrito': 11,
    'cod_barrio': 113,
    'codigo_Postal': 28019
}

# Asignar valores basados en la condición
redpiso_unido.loc[(redpiso_unido['nombre_oficina'] == 'CARABANCHEL-URGEL') & (redpiso_unido['barrio'].isnull()), list(valores.keys())] = list(valores.values())





In [126]:
redpiso_unido.shape

(1467, 51)

In [127]:
columnas_a_eliminar = ["_merge"]

redpiso_unido= redpiso_unido.drop(columns=columnas_a_eliminar)

In [128]:
#tenemos 7 viviendas no ubicada con cod_barrio, las eliminamos del dataframe

redpiso_unido
# Eliminar las filas donde cod_barrio es nulo
redpiso_unido = redpiso_unido[redpiso_unido['cod_barrio'].notnull()]

In [129]:
redpiso_unido.shape

(1460, 50)

In [130]:
redpiso_unido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 0 to 2096
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              1460 non-null   object 
 1   precio              1460 non-null   float64
 2   cantidad_visitas    1460 non-null   object 
 3   fecha_publicacion   1460 non-null   object 
 4   referencia          1460 non-null   object 
 5   m2_constr           1460 non-null   float64
 6   m2_utiles           1079 non-null   object 
 7   dormitorios         1458 non-null   float64
 8   banos               1456 non-null   object 
 9   titularanuncio      967 non-null    object 
 10  descripcion_larga   1454 non-null   object 
 11  amueblado           1 non-null      object 
 12  alarma              3 non-null      object 
 13  num_pisos           16 non-null     object 
 14  num_plantas         20 non-null     object 
 15  puerta_seguridad    18 non-null     object 
 16  acceso_minu

In [131]:
redpiso_unido.columns

Index(['titulo', 'precio', 'cantidad_visitas', 'fecha_publicacion',
       'referencia', 'm2_constr', 'm2_utiles', 'dormitorios', 'banos',
       'titularanuncio', 'descripcion_larga', 'amueblado', 'alarma',
       'num_pisos', 'num_plantas', 'puerta_seguridad', 'acceso_minusvalido',
       'porteria', 'terraza', 'zonas_verdes', 'planta', 'garaje', 'trastero',
       'gastos_comunidad', 'tiene_piscina', 'exterior', 'tipo_fachada',
       'tipo_suelo', 'calefaccion', 'aire_acondicionado', 'antiguedad',
       'ascensor', 'agua_caliente', 'estado', 'emisiones_CO2',
       'consumo_energia', 'armarios', 'orientacion', 'url', 'nombre_oficina',
       'nuda_propiedad', 'web', 'fecha_descarga', 'tipologia', 'calle',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'codigo_Postal'],
      dtype='object')

In [132]:
redpiso_unido[['url']].value_counts(dropna=False)

url                                                                                                                                      
https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-alberto-aguilera-gaztambide-chamberi-madrid-madrid-RP1782024122328             1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902022102747             1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-princesa-de-eboli-sanchinarro-valdefuentes-hortaleza-madrid-madrid-RP1122024126380    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-prieto-urena-hispanoamerica-chamartin-madrid-madrid-RP90201624629                     1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pozas-universidad-centro-madrid-madrid-RP1292024131371                                1
                                                                                                                                            ..
http

In [133]:
redpiso_unido["armarios"].value_counts(dropna=False)

armarios
NaN    879
 2     167
 1     157
 3     121
 4      76
 5      29
 6      19
 8       6
 7       5
 9       1
Name: count, dtype: int64

In [134]:
# Definir la lista de columnas a transformar
columnas_a_transformar = ['aire_acondicionado', 'terraza', 'trastero','garaje','amueblado','alarma','puerta_seguridad','acceso_minusvalido','porteria','zonas_verdes','tiene_piscina','exterior','ascensor']

# Transformar los valores a 1 si tienen algún valor
for columna in columnas_a_transformar:
    redpiso_unido.loc[redpiso_unido[columna].notnull(), columna] = 1

# Mostrar el dataframe actualizado

------------------------------------------

In [136]:
redpiso_unido[['url']].value_counts(dropna=False)

url                                                                                                                                      
https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-alberto-aguilera-gaztambide-chamberi-madrid-madrid-RP1782024122328             1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902022102747             1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-princesa-de-eboli-sanchinarro-valdefuentes-hortaleza-madrid-madrid-RP1122024126380    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-prieto-urena-hispanoamerica-chamartin-madrid-madrid-RP90201624629                     1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pozas-universidad-centro-madrid-madrid-RP1292024131371                                1
                                                                                                                                            ..
http

In [137]:
redpiso_unido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 0 to 2096
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   titulo              1460 non-null   object 
 1   precio              1460 non-null   float64
 2   cantidad_visitas    1460 non-null   object 
 3   fecha_publicacion   1460 non-null   object 
 4   referencia          1460 non-null   object 
 5   m2_constr           1460 non-null   float64
 6   m2_utiles           1079 non-null   object 
 7   dormitorios         1458 non-null   float64
 8   banos               1456 non-null   object 
 9   titularanuncio      967 non-null    object 
 10  descripcion_larga   1454 non-null   object 
 11  amueblado           1 non-null      object 
 12  alarma              3 non-null      object 
 13  num_pisos           16 non-null     object 
 14  num_plantas         20 non-null     object 
 15  puerta_seguridad    18 non-null     object 
 16  acceso_minu

In [1557]:
redpiso_unido.to_csv("red_pisoparaunir.csv",index=False)